In [3]:
import pandas as pd
import numpy as np

import sys
import re
import os
project_path = os.getcwd()

# 建模

## 读入数据

In [102]:
df_model =pd.read_excel(project_path +'/data/result/df_model_data.xlsx')
if 'Unnamed: 0' in df_model.columns:
    df_model = df_model.drop(['Unnamed: 0'], axis=1)

In [103]:
discrete_col = ['gender','保肝药']
continuous_col=[x for x in df_model.columns if x not in discrete_col]
continuous_col.remove('bmd_label')

## 数据归一化

In [104]:
# 防止不同维特征数据差距过大，影响建模效果
for i in continuous_col:
    max_value = df_model[i].max()
    df_model[i]=df_model[i].apply(lambda x: round(x/max_value,3))

In [106]:
df_model.columns

Index(['bmd_label', '谷草转氨酶(干式)', 'MTX_tdm_48h', 'γ-谷氨酰酶(干式)', '红细胞分布宽度',
       'gender', '谷丙转氨酶(干式)', 'RBC平均血红量', '红细胞', 'MTX_tdm_24h', '碱性磷酸酶(干式)',
       '肌酐(干式)', '日剂量', 'RBC血红浓度', 'MTX_tdm_12h', '嗜碱性细胞百分比', 'RBC平均容量',
       '总胆红素(干式)', '淋巴细胞绝对值', 'age', '嗜碱性细胞绝对值', '保肝药'],
      dtype='object')

## 插补数据

In [111]:
# 使用随机森林对缺失值进行插补
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
def missing_value_interpolation(df):
    df = df.reset_index(drop=True)
    # 提取存在缺失值的列名
    missing_list = []
    for i in df.columns:
        if df[i].isnull().sum() > 0:
            missing_list.append(i)
    missing_list_copy = missing_list.copy()
    # 用该列未缺失的值训练随机森林，然后用训练好的rf预测缺失值
    for i in range(len(missing_list)):
        name=missing_list[0]
        df_missing = df[missing_list_copy]
        # 将其他列的缺失值用0表示。
        missing_list.remove(name)
        for j in missing_list:
            df_missing[j]=df_missing[j].astype('str').apply(lambda x: 0 if x=='nan' else x)
        df_missing_is = df_missing[df_missing[name].isnull()]
        df_missing_not = df_missing[df_missing[name].notnull()]
        y = df_missing_not[name]
        x = df_missing_not.drop([name],axis=1)
        # 列出参数列表
        tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
        # 进行参数的搜索组合
        grid = GridSearchCV(RandomForestRegressor(),param_grid=tree_grid_parameter,cv=3)
        #rfr=RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1)
        #根据已有数据去拟合随机森林模型
        grid.fit(x, y)
        rfr = RandomForestRegressor(n_estimators=grid.best_params_['n_estimators'])
        rfr.fit(x, y)
        #预测缺失值
        predict = rfr.predict(df_missing_is.drop([name],axis=1))
        #填补缺失值
        df.loc[df[name].isnull(),name] = predict
    return df

In [112]:
# 插补建模数据
df_model_cb=missing_value_interpolation(df_model)

In [113]:
# 保存插补数据
writer = pd.ExcelWriter(project_path + '/data/result/df_model_data_插补.xlsx')
df_model_cb.to_excel(writer)
writer.save()

## 相关性检测

In [89]:
#  连续变量，spearmanr相关性检验(统计量r);
print('--------------------------计算连续变量的spearmanr相关性系数---------------------------------')
from scipy import stats
t_list = []
p_list = []
q_list = []

for i in continuous_col:
    # 删除连续变量中的<、>号
    df_model_cb[i] = df_model_cb[i].astype('str').apply(lambda x: re.sub(r'<|>', '',x))
    x= df_model_cb[df_model_cb[i].astype('float').notnull()][i]
    y= df_model_cb[df_model_cb[i].astype('float').notnull()]['bmd_label']
    t, p = stats.spearmanr(x,y)
    t = round(t, 2)
    p = round(p, 3)
    q = '斯皮尔曼'
    # print(i, t, p)

    t_list.append(t)
    p_list.append(p)
    q_list.append(q)
df_spearmanr= pd.DataFrame(data={'连续检测指标': continuous_col,
                                't值': t_list,
                                'p值': p_list,
                                '方法': q_list})
df_spearmanr_1 = df_spearmanr[df_spearmanr['p值'] <= 0.05]
df_spearmanr_2 = df_spearmanr[df_spearmanr['p值'] >= 0.05]  # 显著性不成立
df_spearmanr = pd.concat([df_spearmanr_1,df_spearmanr_2], axis=0)

df_spearmanr=df_spearmanr.sort_values(by=['p值'],ascending=False)
df_spearmanr = df_spearmanr.reset_index(drop=True)

writer = pd.ExcelWriter(project_path + '/data/result/df_temp_spearmanr相关性检测.xlsx')
df_spearmanr.to_excel(writer)
writer.save()

--------------------------计算连续变量的spearmanr相关性系数---------------------------------


## 划分数据集

In [107]:
from auto_ml.utils_models import load_ml_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [108]:
from auto_ml import Predictor
# 划分训练集和测试集，比例为8:2
x = df_model_cb.drop(['bmd_label'],axis=1)
y = df_model_cb['bmd_label']

tran_x, test_x, tran_y, test_y = train_test_split(x, y, test_size=0.2, random_state=5)

## 训练集过采样

In [109]:
# 进行过采样
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)

tran_x_sm,tran_y_sm = sm.fit_resample(tran_x,tran_y)

## 训练模型

### xgboost

In [110]:
# 直接使用xgboost和catboost包，而不是auto_ml
import xgboost
import lightgbm
import catboost
from sklearn.metrics import r2_score,precision_score,classification_report,confusion_matrix
# XGBoost模型
xgb_model=xgboost.XGBClassifier(max_depth=5,
                        learning_rate=0.01,
                        n_estimators=500,
                        min_child_weight=0.5,
                        eta=0.1,
                        gamma=0.5,
                        reg_lambda=10,
                        subsample=0.5,
                        colsample_bytree=0.8,
                        nthread=4,
                        scale_pos_weight=1)

# LightGBM模型
# xgb_model=lightgbm.LGBMClassifier(iterations=300, learning_rate=0.2, loss_function='RMSE',random_state=3)
# CatBoost模型
# xgb_model=catboost.CatBoostClassifier(iterations=300, learning_rate=0.2, loss_function='RMSE',random_state=3)
# xgb_model=xgboost.XGBClassifier('eta'=0.1,
#                                 'gamma':0.1,
#                                 'max-depth':0.5,
#                                 'min_child_weighty':3,
#                                 'objective':'binary:logistic'
#                                )
xgb_model.fit(tran_x_sm,tran_y_sm)
predictions=xgb_model.predict(test_x)
# print(predictions)
print(classification_report(test_y, xgb_model.predict(test_x)))

cm2_LogR_model = confusion_matrix(test_y, xgb_model.predict(test_x))
print(cm2_LogR_model) #混肴矩阵

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       129
           1       0.20      0.13      0.16        15

    accuracy                           0.85       144
   macro avg       0.55      0.54      0.54       144
weighted avg       0.83      0.85      0.84       144

[[121   8]
 [ 13   2]]


### Random Forest

In [ ]:
# 随机森林，GBDT
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
# 列出参数列表
tree_grid_parameter = {'n_estimators': list((10, 50, 100, 150, 200))}
# 进行参数的搜索组合
grid = GridSearchCV(RandomForestClassifier(), param_grid=tree_grid_parameter, cv=3)
# grid = GridSearchCV(GradientBoostingClassifier(), param_grid=tree_grid_parameter, cv=3)
# 根据已有数据去拟合随机森林模型
grid.fit(tran_x, tran_y)
rfr = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'])
# rfr = GradientBoostingClassifier(n_estimators=grid.best_params_['n_estimators'])
rfr.fit(tran_x, tran_y)
# 预测缺失值
predictions = rfr.predict(test_x)

### SVR

In [ ]:
# SVR
from sklearn.svm import SVR
svr = SVR(kernel='linear', C=1.25)
svr.fit(tran_x,tran_y)
predictions=svr.predict(test_x)

### KNN

In [ ]:
# KNN训练
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()
knn.fit(tran_x,tran_y)
predictions=knn.predict(test_x)

### linear model

In [ ]:
# Linear回归，Lasso回归，领回归
from sklearn.linear_model import LinearRegression,Lasso,Ridge,LogisticRegression
lcv = LogisticRegression()
lcv.fit(tran_x, tran_y)
predictions = lcv.predict(test_x)

## 计算评价指标

In [114]:
# 计算R2和均方误差MSE
print('-----------------------计算R2和均方误差MSE---------------------------')

from sklearn.metrics import mean_squared_error  # 均方误差
from sklearn.metrics import mean_absolute_error  # 平方绝对误差
from sklearn.metrics import precision_score,recall_score,f1_score  # R square
# 调用

r2 = precision_score(test_y,predictions)
print('r2: ',r2)
mse=recall_score(test_y,predictions)
print('MSE: ',mse)
mae=f1_score(test_y,predictions)
print('MAE: ',mae)

-----------------------计算R2和均方误差MSE---------------------------
r2:  0.2
MSE:  0.13333333333333333
MAE:  0.16


In [115]:
test_y.shape

(144,)

In [116]:
df_predictions= pd.DataFrame(data={'真实值':test_y,'预测值':predictions})
writer = pd.ExcelWriter(project_path + '/data/result/df_model_测试集结果.xlsx')
df_predictions.to_excel(writer)
writer.save()

## 画图

In [ ]:
# 判断文件路径是否存在，如果不存在则创建该路径
def mkdir(path):
    folder = os.path.exists(path)
    if not folder:  # 判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)  # makedirs 创建文件时如果路径不存在会创建这个路径

In [ ]:
# 画图
print('-----------------------画图---------------------------')
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import rc
rc('mathtext', default='regular')

# 散点图
# axis设置坐标轴的范围
# plt.axis([-20, 20, 0, 200])
# x为x轴中坐标x的值，y为y轴中坐标y的值，x与y都是长度相同的数组序列，color为点的颜色，marker为散点的形状，
# 折线图刻度调小，要不然点都堆到一块了
ax = plt.gca()
ax.set_xlim(0,10)
ax.set_ylim(0,10)
# plt.scatter(range(len(test_y)),test_y,c='r')
plt.scatter(test_y,predictions,c='b')
# 红色参照线
plt.plot(list(range(test_y.shape[0])), list(range(test_y.shape[0])),color='r')
# plt.plot(list(range(30)), list(range(30)),color='r')
plt.xlabel('Number of Events(unit)')
plt.ylabel('MTX Bone Suppression')
# plt.show()
# 判断图片保存路径是否存在，否则创建
jpg_path = project_path + "/jpg"
mkdir(jpg_path)
plt.savefig(jpg_path + "/他克莫司血药浓度测试集散点图v2.0.jpg", dpi=300)
plt.clf()  # 删除前面所画的图

In [ ]:
'''
# 重要性
import catboost,xgboost
model_boost=xgboost.XGBRegressor()
model_boost.fit(tran_x,tran_y)
importance = model_boost.feature_importances_
print(tran_x.columns)
print(importance)

df_importance= pd.DataFrame(data={'特征':tran_x.columns,'重要性评分':importance})
df_importance['重要性评分']=df_importance['重要性评分'].apply(lambda x: round(x,3))
df_importance=df_importance.sort_values(['重要性评分'],ascending=False)
df_importance=df_importance.reset_index(drop=True)
writer = pd.ExcelWriter(project_path + '/result/df_19_模型重要性评分.xlsx')
df_importance.to_excel(writer)
writer.save()

# SHAP图
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  ##绘图显示中文
mpl.rcParams['axes.unicode_minus'] = False
from matplotlib import rc
rc('mathtext', default='regular')

import xgboost as xgb
import shap
shap.initjs()  # notebook环境下，加载用于可视化的JS代码

importance_list_10 = list(df_importance['特征'])[:10]
tran_x=tran_x[importance_list_10]
shap_model = xgb.XGBRegressor(max_depth=4, learning_rate=0.05, n_estimators=300)
shap_model.fit(tran_x, tran_y)

explainer = shap.TreeExplainer(shap_model)
shap_values = explainer.shap_values(tran_x)  # 传入特征矩阵X，计算SHAP值
# print(shap_values)
# summarize the effects of all the features
shap.summary_plot(shap_values, tran_x, plot_size=(20,12))
# 保存各个变量的shape值的和
df_shap_values=pd.DataFrame(shap_values)
shap_list=[]
for i in range(df_shap_values.shape[1]):
    df_shap_values.iloc[:,i] = df_shap_values.iloc[:,i].apply(lambda x: abs(x))
for j in range(df_shap_values.shape[1]):
    feature_mean = df_shap_values.iloc[:,j].mean()
    feature_mean = round(feature_mean, 2)
    shap_list.append(feature_mean)

df_shap = pd.DataFrame({'features':list(tran_x.columns),'shap值':shap_list})
df_shap = df_shap.sort_values(by=['shap值'], ascending=False)
df_shap = df_shap.reset_index(drop=True)

writer = pd.ExcelWriter(project_path + '/result/df_20_shap值排序.xlsx')
df_shap.to_excel(writer)
writer.save()
'''